#### Google Palm LLM & API key setup

In [182]:
import os
# --- 1) LLM: OpenAI (chat model) ---
#GoolePalm -> ChatOpenAI, HuggingFace -> OpenAIEmbeddings
from langchain_openai import ChatOpenAI, OpenAIEmbeddings    #Change


# Instantiate the LLM (reads OPENAI_API_KEY from env)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)

# Quick check
#humor = llm.invoke("Write a short humor for 40yrs old ladies").content
#print(humor)

#### Connect with database and ask some basic questions

In [183]:

from langchain_community.utilities import SQLDatabase   #ss
from langchain_experimental.sql import SQLDatabaseChain   #ss
from langchain_community.agent_toolkits import create_sql_agent      ##Change
from langchain_community.agent_toolkits import SQLDatabaseToolkit    #Change

db_user = "root"
db_password = "root061783"
db_host = "localhost"
db_name = "go_tshirts"


# Requires `pymysql`
db = SQLDatabase.from_uri(
    f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",
    sample_rows_in_table_info=3
)
table_info = db.table_info
print(table_info)  # a helpful schema snapshot
#####++++
# Create the SQL agent and toolkit
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
agent_executor = create_sql_agent(llm=llm, toolkit=toolkit, verbose=True) #db_chain->agent_executor




CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))
)COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount
11	100	27.00
12	11	39.00
13	3	20.00
*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price

Above is the correct answer 👍🏼

In [184]:

qns1 = agent_executor.invoke({
    "input": "How many t-shirts do we have left for Nike in extra small size and white color?"})

# Optional: print just the text answer
print("Q1:", qns1.get("output", qns1))





> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables  
Action Input: ""  discounts, t_shirtsI need to check the schema of the `t_shirts` table to find the relevant columns for querying the inventory of Nike t-shirts in extra small size and white color.  
Action: sql_db_schema  
Action Input: "t_shirts"  
CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock_quantity
1	Nike	Black	XL	24	18
2	Levi	White	L	26	71
3	Van Huesen	Red	L	35	75
*/To find out how many extra small white Nike t-shirts are left, I need to construct a qu

In [185]:
qns1 = agent_executor.invoke({ "input": "SELECT stock_quantity FROM t_shirts WHERE brand = 'Nike' AND size = 'XS' AND color = 'White';"})



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables  
Action Input: ""  discounts, t_shirtsI need to check the schema of the `t_shirts` table to understand its structure and confirm the columns available for querying.  
Action: sql_db_schema  
Action Input: "t_shirts"  
CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock_quantity
1	Nike	Black	XL	24	18
2	Levi	White	L	26	71
3	Van Huesen	Red	L	35	75
*/The `t_shirts` table contains the necessary columns for my query. I can now construct the SQL query to find the stock qua

In [188]:

qns2 = agent_executor.invoke({ "input": "How much is the price of the inventory for all small size t-shirts?"})
print("Q2:", qns2.get("output", qns2))




> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables  
Action Input: ""  discounts, t_shirtsI need to check the schema of the `t_shirts` table to find out the relevant columns for querying the price and size of the t-shirts.  
Action: sql_db_schema  
Action Input: "t_shirts"  
CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock_quantity
1	Nike	Black	XL	24	18
2	Levi	White	L	26	71
3	Van Huesen	Red	L	35	75
*/I need to construct a query to calculate the total price of all small size t-shirts. The relevant columns are `size

It made a mistake here. The price is actually the price per unit but in real life database columns will not have perfect names. We need to tell it somehow that price is price per unit and the actual query should be,

SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'

In [186]:
qns2 = agent_executor.invoke({"input" : "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE size = 'S';"})




> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables  
Action Input: ""  discounts, t_shirtsI need to check the schema of the `t_shirts` table to understand its structure and confirm the columns available for querying.  
Action: sql_db_schema  
Action Input: "t_shirts"  
CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock_quantity
1	Nike	Black	XL	24	18
2	Levi	White	L	26	71
3	Van Huesen	Red	L	35	75
*/The `t_shirts` table contains the columns `price` and `stock_quantity`, which I need for my query. The query I want to ex

we will use qns2 value later on in this notebook. So hold on for now and let's check another query

In [32]:
qns3 = agent_executor.invoke({"input": "If we have to sell all the Levi’s T-shirts today with discounts applied. How much revenue our store will generate (post discounts)?"})
print("Q3:", qns3.get("output", qns3))






> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables  
Action Input: ""  discounts, t_shirtsI need to check the schema of the `t_shirts` table to understand its structure and see how to calculate the revenue from selling Levi's T-shirts with discounts applied.  
Action: sql_db_schema  
Action Input: "t_shirts"  
CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock_quantity
1	Levi	Blue	XS	40	24
2	Nike	Blue	S	29	39
3	Van Huesen	Red	XS	35	21
*/To calculate the revenue from selling all Levi's T-shirts with discounts applied

Above, it returned a wrong query which generated an error during query execution. It thinks discount
table would have start and end date which is normally true but in our table there is no start or end date column.
One thing we can do here is run the query directly.

In [137]:
#sql_code = """
#select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
#(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
#group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
# """

#qns3 = agent_executor.invoke(sql_code)

It produced a correct answer when explicit query was given. 17462 is the total revenue without discounts. The total discount is 736.6. Hence revenue post discount is 17462-736.6=16725.4

Now this is not much interesting because what is the point of giving it the ready made query? Well, we will use this same query later on for few shot learning

In [189]:
qns4 = agent_executor.invoke("SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'")



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables  
Action Input: ""  discounts, t_shirtsI need to check the schema of the `t_shirts` table to understand its structure and confirm the columns available for my query.  
Action: sql_db_schema  
Action Input: "t_shirts"  
CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock_quantity
1	Nike	Black	XL	24	18
2	Levi	White	L	26	71
3	Van Huesen	Red	L	35	75
*/I can see that the `t_shirts` table has the necessary columns: `price` and `stock_quantity`. I will now construct the que

In [103]:
qns5 = agent_executor.invoke("How many white color Levi's t shirts we have available?")



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables  
Action Input: ""  discounts, t_shirtsThe relevant table for t-shirts is "t_shirts". I will now check the schema of the "t_shirts" table to find the relevant columns for my query.  
Action: sql_db_schema  
Action Input: "t_shirts"  
CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock_quantity
1	Levi	Blue	XS	40	24
2	Nike	Blue	S	29	39
3	Van Huesen	Red	XS	35	21
*/To find out how many white color Levi's t-shirts are available, I need to query the "t_shirts" table for th

Once again above is the wrong answer. We need to use SUM(stock_quantity). Let's run the query explicitly. We will use the result of this query later on in the notebook

In [190]:
qns5 = agent_executor.invoke("SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'")



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables  
Action Input: ""  discounts, t_shirtsI need to check the schema of the `t_shirts` table to understand its structure and confirm the columns available for querying.  
Action: sql_db_schema  
Action Input: "t_shirts"  
CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock_quantity
1	Nike	Black	XL	24	18
2	Levi	White	L	26	71
3	Van Huesen	Red	L	35	75
*/I can see that the `t_shirts` table has the columns I need to execute the query. The relevant columns are `brand`, `color

#### Few shot learning

We will use few shot learning to fix issues we have seen so far

In [191]:
few_shots = [
    {
        "Question": "How many t-shirts do we have left for Nike in XS size and white color?",
        "SQLQuery": "SELECT SUM(stock_quantity) FROM t_shirts WHERE brand='Nike' AND color='White' AND size='XS'",
        "SQLResult": "Result of the SQL query",
        "Answer": str(qns1),
    },
    {
        "Question": "How much is the total price of the inventory for all S-size t-shirts?",
        "SQLQuery": "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE size='S'",
        "SQLResult": "Result of the SQL query",
        "Answer": str(qns2),
    },
    {
        "Question": "If we sell all the Levi’s T-shirts today (no discounts), how much revenue do we generate?",
        "SQLQuery": "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand='Levi'",
        "SQLResult": "Result of the SQL query",
        "Answer": str(qns4),
    },
    {
        "Question": "How many white color Levi's shirts do I have?",
        "SQLQuery": "SELECT SUM(stock_quantity) FROM t_shirts WHERE brand='Levi' AND color='White'",
        "SQLResult": "Result of the SQL query",
        "Answer": str(qns5),
    },
]

### Creating Semantic Similarity Based example selector

- create embedding on the few_shots
- Store the embeddings in Chroma DB
- Retrieve the the top most Semantically close example from the vector store

In [192]:

####chat 1) Keep your original few_shots, then add "input" = "Question" per example
few_shots_with_input = [{**ex, "input": ex["Question"]} for ex in few_shots]


#HuggingFaceEmbeddings -> OpenAI
from langchain_community.vectorstores import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector #ss_UPDATE
from langchain_core.prompts import FewShotPromptTemplate  #ss
from langchain_core.prompts import PromptTemplate  #ss
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _mysql_prompt #ss


embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Recommended helper: creates the vector store for you
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples=few_shots_with_input,           # list[dict[str, str]]     ####chat
    embeddings=embeddings,        # any LangChain Embeddings
    vectorstore_cls=Chroma,       # which store to use
    k=2,                          # how many examples to retrieve
    input_keys=["input"],      # which field(s) to match on       ####chat
)

# Try it:
selected = example_selector.select_examples(
    {"input": "How many Adidas T-shirts do I have left in my store?"}    ####chat
)
print("Selected few-shot examples:", selected)


Selected few-shot examples: [{'Question': 'How many t-shirts do we have left for Nike in XS size and white color?', 'SQLQuery': "SELECT SUM(stock_quantity) FROM t_shirts WHERE brand='Nike' AND color='White' AND size='XS'", 'Answer': 'The stock quantity of Nike t-shirts in size XS and color White is 17.', 'SQLResult': 'Result of the SQL query'}, {'Answer': 'The stock quantity of Nike t-shirts in size XS and color White is 17.', 'SQLQuery': "SELECT SUM(stock_quantity) FROM t_shirts WHERE brand='Nike' AND color='White' AND size='XS'", 'SQLResult': 'Result of the SQL query', 'Question': 'How many t-shirts do we have left for Nike in XS size and white color?'}]


In [193]:
### my sql based instruction prompt

mysql_prompt = """You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: Query to run with no pre-amble
SQLResult: Result of the SQLQuery
Answer: Final answer here

No pre-amble.

"""
prompt_suffix = """Only use the following tables:
CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))
)COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount

*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4 ENGINE=InnoDB

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price	stock_quantity
1	Levi	Blue	XS	40	24
2	Nike	Blue	S	29	39
3	Van Huesen	Red	XS	35	21
*/

Question: {input}"""



### Setting up PromptTemplete using input variables

In [194]:

# --- How examples are rendered in the prompt ---
example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult", "Answer"],
    template="\nQuestion: {Question}\nSQLQuery:{SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",
)


In [195]:
# --- FewShot prompt (prefix + selected examples + suffix) ---
few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector, ##Establish the asscociation between your LLM and Vector database
    example_prompt=example_prompt,
    prefix=mysql_prompt,
    suffix=prompt_suffix,  # prompt_suffix__PROMPT_SUFFIX__expects {input}, {table_info}, {top_k}
    input_variables=["input", "table_info", "top_k"]#These variables are used in the prefix and suffix
)

new_chain = SQLDatabaseChain.from_llm(
    llm=llm,
    db=db,
    prompt=few_shot_prompt,
    verbose=True,
    use_query_checker=True,  # catches common SQL issues
    top_k=10,                # default LIMIT size passed into the prompt
    #return_sql=True,     ###Error Check
)



In [196]:

# --- Try it ---
new_chain("How many white color Nike shirt I have?")





> Entering new SQLDatabaseChain chain...
How many white color Nike shirt I have?
SQLQuery:```sql
SELECT SUM(stock_quantity) FROM t_shirts WHERE brand='Nike' AND color='White'
```

ProgrammingError: (pymysql.err.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '```sql\nSELECT SUM(stock_quantity) FROM t_shirts WHERE brand='Nike' AND color='Wh' at line 1")
[SQL: ```sql
SELECT SUM(stock_quantity) FROM t_shirts WHERE brand='Nike' AND color='White'
```]
(Background on this error at: https://sqlalche.me/e/20/f405)

Now this is working ok. Previously for this same question it was giving wrong answer because it did not use SUM clause around stock_quantity column

In [181]:
new_chain("How much is the price of the inventory for all small size t-shirts?")



> Entering new SQLDatabaseChain chain...
How much is the price of the inventory for all small size t-shirts?
SQLQuery:```sql
SELECT SUM(price * stock_quantity) FROM t_shirts WHERE size='S'
```

ProgrammingError: (pymysql.err.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '```sql\nSELECT SUM(price * stock_quantity) FROM t_shirts WHERE size='S'\n```' at line 1")
[SQL: ```sql
SELECT SUM(price * stock_quantity) FROM t_shirts WHERE size='S'
```]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [ ]:
new_chain("How much is the price of all white color levi t shirts?")

In [ ]:
new_chain("If we have to sell all the Nike’s T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?")

In [ ]:
new_chain("If we have to sell all the Van Heuson T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?")

In [ ]:
new_chain.invoke('How much revenue  our store will generate by selling all Van Heuson TShirts without discount?')